In [0]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow2/text_matching/snli/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!pip install tensorflow-gpu==2.0.0-rc0

In [0]:
import tensorflow as tf
import numpy as np
import pprint
import logging
import time
import math

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 2.0.0-rc0
GPU Enabled: True


In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip()
      word2idx[line] = i
  return word2idx

In [0]:
# stream data from text files
def data_generator(f_path, word2idx):
  label2idx = {'neutral': 0, 'entailment': 1, 'contradiction': 2,}
  with open(f_path) as f:
    print('Reading', f_path)
    for line in f:
      line = line.rstrip()
      label, text1, text2 = line.split('\t')
      if label == '-':
        continue
      text1 = [word2idx.get(w, len(word2idx)) for w in text1.split()]
      text2 = [word2idx.get(w, len(word2idx)) for w in text2.split()]
      yield ((text1, text2), label2idx[label])


def dataset(is_training, params):
  _shapes = (([None], [None]), ())
  _types = ((tf.int32, tf.int32), tf.int32)
  _pads = ((0, 0), -1)
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params['word2idx']),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
  
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params['word2idx']),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
  
  return ds

In [0]:
class RE2(tf.keras.Model):
  def __init__(self, params: dict):
    super().__init__()
    
    self.pretrained_embedding = tf.Variable(np.load(params['embedding_path']), name='pretrained_embedding')
    self.embed_dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    
    self.birnn = [tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
        params['hidden_size'], return_sequences=True), name='birnn_%d'%(i+1)) for i in range(params['num_blocks'])]
    self.enc_dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    
    self.align_t = self.add_weight(name='temperature',
                                   shape=(),
                                   trainable=True,
                                   initializer=tf.initializers.constant(math.sqrt(1/params['hidden_size'])))
    self.align_dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.align_fc1 = [tf.keras.layers.Dense(params['hidden_size'], params['activation'], name='align_fc1_%d'%(i+1)) for i in range(params['num_blocks'])]
    self.align_fc2 = [tf.keras.layers.Dense(params['hidden_size'], params['activation'], name='align_fc2_%d'%(i+1)) for i in range(params['num_blocks'])]
      
    self.fusion_fc1 = [tf.keras.layers.Dense(params['hidden_size'], params['activation'], name='fusion_fc1_%d'%(i+1)) for i in range(params['num_blocks'])]
    self.fusion_fc2 = [tf.keras.layers.Dense(params['hidden_size'], params['activation'], name='fusion_fc2_%d'%(i+1)) for i in range(params['num_blocks'])]
    self.fusion_fc3 = [tf.keras.layers.Dense(params['hidden_size'], params['activation'], name='fusion_fc3_%d'%(i+1)) for i in range(params['num_blocks'])]
    self.fusion_fc4 = [tf.keras.layers.Dense(params['hidden_size'], params['activation'], name='fusion_fc4_%d'%(i+1)) for i in range(params['num_blocks'])]
    self.fusion_dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    
    self.out_drop1 = tf.keras.layers.Dropout(params['dropout_rate'])
    self.out_fc = tf.keras.layers.Dense(params['hidden_size'], params['activation'], name='out_fc')
    self.out_drop2 = tf.keras.layers.Dropout(params['dropout_rate'])
    self.out_linear = tf.keras.layers.Dense(params['num_labels'], name='out_linear')
    
  
  
  def call(self, inputs, training=False):
    x1, x2 = inputs
    
    if x1.dtype != tf.int32:
      x1 = tf.cast(x1, tf.int32)
    if x2.dtype != tf.int32:
      x2 = tf.cast(x2, tf.int32)
    
    batch_sz = tf.shape(x1)[0]
    
    mask1 = tf.sign(x1)
    mask2 = tf.sign(x2)
    
    x1 = self.embedding(x1, training=training)
    x2 = self.embedding(x2, training=training)
    
    res_x1, res_x2 = x1, x2
    
    for i in range(params['num_blocks']):
      
      if i > 0:
        x1 = self.connection(x1, res_x1, i)
        x2 = self.connection(x2, res_x2, i)
        res_x1, res_x2 = x1, x2
    
      x1_enc = self.encoding(x1, mask1, i, training=training)
      x2_enc = self.encoding(x2, mask2, i, training=training)
    
      x1 = tf.concat((x1, x1_enc), -1)
      x2 = tf.concat((x2, x2_enc), -1)
      
      align_1, align_2 = self.alignment(x1, x2, mask1, mask2, i, training=training)
      
      x1 = self.fusion(x1, align_1, i, training=training)
      x2 = self.fusion(x2, align_2, i, training=training)
    
    x1 = self.pooling(x1, mask1)
    x2 = self.pooling(x2, mask2)
    
    x = self.prediction(x1, x2, training=training)
    
    return x
  
  
  def embedding(self, x, training):
    x = tf.nn.embedding_lookup(self.pretrained_embedding, x)
    x = self.embed_dropout(x, training=training)
    return x
  
  
  def connection(self, x, res, i):
    if i == 1:
      x = tf.concat((res, x), -1)
    elif i > 1:
      hidden_size = x.shape[-1]
      x = (res[:, :, -hidden_size:] + x) * tf.math.sqrt(0.5)
      x = tf.concat((res[:, :, :-hidden_size], x), -1)
    return x
    
    
  def encoding(self, x, mask, i, training):
    mask = tf.cast(tf.expand_dims(mask, -1), tf.float32)
    x *= mask
    x = self.birnn[i](x)
    x = self.enc_dropout(x)
    return x
  
  
  def alignment(self, x1, x2, mask1, mask2, i, training):
    mask1 = tf.cast(tf.expand_dims(mask1, -1), tf.float32)
    mask2 = tf.cast(tf.expand_dims(mask2, -1), tf.float32)
    
    x1_ = self.align_fc1[i](self.align_dropout(x1, training=training))
    x2_ = self.align_fc2[i](self.align_dropout(x2, training=training))
    align = tf.matmul(x1_, x2_, transpose_b=True) * self.align_t
    mask = tf.matmul(mask1, mask2, transpose_b=True)
    align = mask * align + (1 - mask) * tf.float32.min
    align_1 = tf.nn.softmax(align, 1)
    align_2 = tf.nn.softmax(align, 2)
    
    x2 = tf.matmul(align_1, x1, transpose_a=True)
    x1 = tf.matmul(align_2, x2)
    return x1, x2
  
  
  def fusion(self, x, align, i, training):
    x = tf.concat([self.fusion_fc1[i](tf.concat((x, align), -1)),
                   self.fusion_fc2[i](tf.concat((x, x - align), -1)),
                   self.fusion_fc3[i](tf.concat((x, x * align), -1))], -1)
    x = self.fusion_dropout(x, training=training)
    x = self.fusion_fc4[i](x)
    return x
  
  
  def pooling(self, x, mask):
    mask = tf.cast(tf.expand_dims(mask, -1), tf.float32)
    return tf.reduce_max(x * mask, 1)
  
  
  def prediction(self, x1, x2, training):
    x = tf.concat((x1, x2, x1 * x2, x1 - x2), -1)
    x = self.out_drop1(x, training=training)
    x = self.out_fc(x)
    x = self.out_drop2(x, training=training)
    x = self.out_linear(x)
    return x

In [0]:
params = {
    'model_dir': '../model/re2_birnn',
    'log_path': '../log/re2_birnn.txt',
    'train_path': '../data/train.txt',
    'test_path': '../data/test.txt',
    'embedding_path': '../vocab/word.npy',
    'vocab_path': '../vocab/word.txt',
    'batch_size': 32,
    'buffer_size': 200000,
    'num_blocks': 2,
    'dropout_rate': 0.2,
    'hidden_size': 150,
    'activation': tf.nn.relu,
    'lr': 4e-4,
    'clip_norm': 5.,
    'num_labels': 3,
    'eval_steps': 550152//32//10,
    'num_patience': 7,
}

In [0]:
def is_descending(history: list):
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True  

In [9]:
params['word2idx'] = get_vocab(params['vocab_path'])
params['vocab_size'] = len(params['word2idx']) + 1

model = RE2(params)
model.build([[None, None], [None, None]])
pprint.pprint([(v.name, v.shape) for v in model.trainable_variables])

decay_lr = tf.optimizers.schedules.ExponentialDecay(params['lr'], 1000, 0.99)
optim = tf.optimizers.Adam(params['lr'])
global_step = 0

history_acc = []
best_acc = .0

t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)

while True:
  # TRAINING
  for ((text1, text2), label) in dataset(is_training=True, params=params):
    with tf.GradientTape() as tape:
      logits = model((text1, text2), training=True)
      loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label, logits=logits)
      loss = tf.reduce_mean(loss)
      
    optim.lr.assign(decay_lr(global_step))
    grads = tape.gradient(loss, model.trainable_variables)
    grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    optim.apply_gradients(zip(grads, model.trainable_variables))
    
    if global_step % 50 == 0:
      logger.info("Step {} | Loss: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
          global_step, loss.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    global_step += 1
  
    if global_step % params['eval_steps'] == 0:
      # EVALUATION
      m = tf.keras.metrics.Accuracy()

      for ((text1, text2), label) in dataset(is_training=False, params=params):
        logits = model((text1, text2), training=False)
        y_pred = tf.argmax(logits, axis=-1)
        m.update_state(y_true=label, y_pred=y_pred)

      acc = m.result().numpy()
      logger.info("Evaluation: Testing Accuracy: {:.3f}".format(acc))
      history_acc.append(acc)

      if acc > best_acc:
        best_acc = acc
        model.save_weights('re2_birnn.ckpt')
      logger.info("Best Accuracy: {:.3f}".format(best_acc))

      if len(history_acc) > params['num_patience'] and is_descending(history_acc):
        logger.info("Testing Accuracy not improved over {} epochs, Early Stop".format(params['num_patience']))
        break

[('birnn_1/forward_lstm/kernel:0', TensorShape([300, 600])),
 ('birnn_1/forward_lstm/recurrent_kernel:0', TensorShape([150, 600])),
 ('birnn_1/forward_lstm/bias:0', TensorShape([600])),
 ('birnn_1/backward_lstm/kernel:0', TensorShape([300, 600])),
 ('birnn_1/backward_lstm/recurrent_kernel:0', TensorShape([150, 600])),
 ('birnn_1/backward_lstm/bias:0', TensorShape([600])),
 ('birnn_2/forward_lstm_1/kernel:0', TensorShape([450, 600])),
 ('birnn_2/forward_lstm_1/recurrent_kernel:0', TensorShape([150, 600])),
 ('birnn_2/forward_lstm_1/bias:0', TensorShape([600])),
 ('birnn_2/backward_lstm_1/kernel:0', TensorShape([450, 600])),
 ('birnn_2/backward_lstm_1/recurrent_kernel:0', TensorShape([150, 600])),
 ('birnn_2/backward_lstm_1/bias:0', TensorShape([600])),
 ('align_fc1_1/kernel:0', TensorShape([600, 150])),
 ('align_fc1_1/bias:0', TensorShape([150])),
 ('align_fc1_2/kernel:0', TensorShape([750, 150])),
 ('align_fc1_2/bias:0', TensorShape([150])),
 ('align_fc2_1/kernel:0', TensorShape([600, 

W0827 01:02:34.241458 140169079596928 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/clip_ops.py:301: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
I0827 01:02:34.444754 140169079596928 interactiveshell.py:2882] Step 0 | Loss: 1.1624 | Spent: 39.6 secs | LR: 0.000400
I0827 01:02:46.402678 140169079596928 interactiveshell.py:2882] Step 50 | Loss: 1.0562 | Spent: 12.0 secs | LR: 0.000400
I0827 01:02:58.878019 140169079596928 interactiveshell.py:2882] Step 100 | Loss: 1.1504 | Spent: 12.5 secs | LR: 0.000400
I0827 01:03:11.051404 140169079596928 interactiveshell.py:2882] Step 150 | Loss: 0.9032 | Spent: 12.2 secs | LR: 0.000399
I0827 01:03:23.249361 140169079596928 interactiveshell.py:2882] Step 200 | Loss: 1.0142 | Spent: 12.2 secs | LR: 0.000399
I0827 01:03:35.385014 140169079596928 interactiveshel

Reading ../data/test.txt


I0827 01:10:01.394948 140169079596928 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.766
I0827 01:10:01.562734 140169079596928 interactiveshell.py:2882] Best Accuracy: 0.766
I0827 01:10:09.457973 140169079596928 interactiveshell.py:2882] Step 1750 | Loss: 0.7427 | Spent: 46.4 secs | LR: 0.000393
I0827 01:10:21.730384 140169079596928 interactiveshell.py:2882] Step 1800 | Loss: 1.0084 | Spent: 12.3 secs | LR: 0.000393
I0827 01:10:33.731327 140169079596928 interactiveshell.py:2882] Step 1850 | Loss: 0.8283 | Spent: 12.0 secs | LR: 0.000393
I0827 01:10:45.865643 140169079596928 interactiveshell.py:2882] Step 1900 | Loss: 0.6718 | Spent: 12.1 secs | LR: 0.000392
I0827 01:10:58.061648 140169079596928 interactiveshell.py:2882] Step 1950 | Loss: 0.7655 | Spent: 12.2 secs | LR: 0.000392
I0827 01:11:10.101323 140169079596928 interactiveshell.py:2882] Step 2000 | Loss: 0.7166 | Spent: 12.0 secs | LR: 0.000392
I0827 01:11:22.296489 140169079596928 interactiveshell.py:2882] Step 2050 | L

Reading ../data/test.txt


...

Reading ../data/test.txt


I0827 10:08:14.399233 140169079596928 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.877
I0827 10:08:14.596940 140169079596928 interactiveshell.py:2882] Best Accuracy: 0.877
I0827 10:08:22.985298 140169079596928 interactiveshell.py:2882] Step 123800 | Loss: 0.1769 | Spent: 46.3 secs | LR: 0.000115
I0827 10:08:35.518418 140169079596928 interactiveshell.py:2882] Step 123850 | Loss: 0.1870 | Spent: 12.5 secs | LR: 0.000115
I0827 10:08:47.827455 140169079596928 interactiveshell.py:2882] Step 123900 | Loss: 0.1235 | Spent: 12.3 secs | LR: 0.000115
I0827 10:09:00.136824 140169079596928 interactiveshell.py:2882] Step 123950 | Loss: 0.4419 | Spent: 12.3 secs | LR: 0.000115
I0827 10:09:12.764320 140169079596928 interactiveshell.py:2882] Step 124000 | Loss: 0.2300 | Spent: 12.6 secs | LR: 0.000115
I0827 10:09:25.141168 140169079596928 interactiveshell.py:2882] Step 124050 | Loss: 0.2463 | Spent: 12.4 secs | LR: 0.000115
I0827 10:09:37.637142 140169079596928 interactiveshell.py:2882] S

Reading ../data/test.txt


I0827 10:15:53.814267 140169079596928 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.876
I0827 10:15:53.815483 140169079596928 interactiveshell.py:2882] Best Accuracy: 0.877
I0827 10:15:57.247464 140169079596928 interactiveshell.py:2882] Step 125500 | Loss: 0.4634 | Spent: 45.0 secs | LR: 0.000113
I0827 10:16:09.548271 140169079596928 interactiveshell.py:2882] Step 125550 | Loss: 0.0417 | Spent: 12.3 secs | LR: 0.000113


KeyboardInterrupt: ignored